In [64]:
# Fazendo os Imports

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [65]:
# Importando o DataFrame

df = pd.read_csv("../input/gas-prices-in-brazil/2004-2021.tsv",sep="\t")

df.head()

In [66]:
df.describe()

In [67]:
#Análise de número de postos por Estados em ordem decrescente

dfnumpostos = df.groupby(['ESTADO']).max()['NÚMERO DE POSTOS PESQUISADOS']
print(dfnumpostos.sort_values(ascending = False))

ax = sns.histplot(data=df , x="ESTADO", y="NÚMERO DE POSTOS PESQUISADOS")
plt.xticks(rotation=270)


In [68]:
#Análise de preço médio de revenda em ordem decrescente

dfpreco = df.query("PRODUTO == 'GASOLINA COMUM'").groupby(['ESTADO']).mean()['PREÇO MÉDIO REVENDA'].sort_values(ascending = False).reset_index()
dfpreco.sort_values('PREÇO MÉDIO REVENDA', ascending=False)
dfpreco

In [69]:
df['PRODUTO'] = df['PRODUTO'].str.replace('OLEO DIESEL S10', 'ÓLEO DIESEL S10')
df['PRODUTO'] = df['PRODUTO'].str.replace('OLEO DIESEL', 'ÓLEO DIESEL')
df["PRODUTO"].unique()

In [70]:
# Preço médio de revenda dos combustiveis 
dfmediapreco = df.groupby(["PRODUTO"]).mean()['PREÇO MÉDIO REVENDA']

print("Preço Médio de Revenda de 2004 a 2021:\n",dfmediapreco)

In [71]:
df['DATA FINAL'] =  pd.to_datetime(df['DATA FINAL'], format='%Y/%m/%d')
df['ANO'] = df['DATA FINAL'].dt.year
df['MES'] = df['DATA FINAL'].dt.month
df['ANO/MES']=pd.to_datetime({'day': 1,'month': df['MES'],'year': df['ANO']})

In [72]:
# Análise gráfica do preço médio de revenda dos combustíveis

dt1=df.query("PRODUTO != 'GLP' & PRODUTO != 'GASOLINA ADITIVADA'")
dt1=dt1[['PRODUTO','ANO/MES','PREÇO MÉDIO REVENDA']].groupby(['PRODUTO','ANO/MES']).mean().sort_values(by=['PRODUTO','ANO/MES'], ascending=True)
dt2=dt1.pivot_table(index=['ANO/MES'], columns=['PRODUTO'], values=['PREÇO MÉDIO REVENDA'])
dt2.plot(kind='line',figsize=(11,7)).get_legend().set_bbox_to_anchor((1, 1))

In [73]:
# Análise gráfica do preço médio de revenda na Região Sudeste

df1=df.query("PRODUTO == 'ETANOL HIDRATADO' & REGIÃO=='SUDESTE'")
df1=df1[['ESTADO','ANO/MES','PREÇO MÉDIO REVENDA']].groupby(['ESTADO','ANO/MES']).mean().sort_values(by=['ESTADO','ANO/MES'], ascending=True)
df2=df1.pivot_table(index=['ANO/MES'], columns=['ESTADO'], values=['PREÇO MÉDIO REVENDA'])

df2.plot(kind='line',figsize=(11,7)).get_legend().set_bbox_to_anchor((1, 1))

In [74]:
np.random.seed(100)

In [75]:
# Treino e Teste

dfgascomum = df.query('PRODUTO =="GASOLINA COMUM"')

X = dfgascomum.drop(['PREÇO MÉDIO REVENDA','DATA INICIAL','DATA FINAL','REGIÃO', 'ESTADO','PRODUTO','UNIDADE DE MEDIDA','MARGEM MÉDIA REVENDA','PREÇO MÉDIO DISTRIBUIÇÃO','DESVIO PADRÃO DISTRIBUIÇÃO','PREÇO MÍNIMO DISTRIBUIÇÃO','PREÇO MÁXIMO DISTRIBUIÇÃO','COEF DE VARIAÇÃO DISTRIBUIÇÃO','ANO/MES'], axis=1)

y = dfgascomum['PREÇO MÉDIO REVENDA']

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 42,test_size=0.30)

In [76]:
# Criando o objeto reg_all
reg_all = LinearRegression()

# Aplicando aos dados
reg_all.fit(X_train, y_train)

# Predicao
y_pred = reg_all.predict(X_test)

#  R^2 and RMSE
print("R^2: {}".format(reg_all.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))